<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/computational_methods/NeuralNetworkPrimer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision as tv
from plotly.subplots import make_subplots
import sklearn
import torchsummary

In [2]:
x = torch.linspace(0,10, 25).unsqueeze(1)
y= torch.sin(x)
x_norm = (x-x.mean())/x.std()

In [3]:
lr=1e-1

w1=torch.distributions.Normal(0, 1/20**0.5).sample((1,20)).requires_grad_()
b1=torch.zeros((1,20),requires_grad=True)

for i in range(20000):
  o1=torch.tanh(x_norm@w1+b1)
  predict=torch.sum(o1,dim=1)
  loss = torch.mean((predict-y.squeeze())**2)
  if i % 1000 ==0:
    print(i, loss)
  loss.backward()
  with torch.no_grad():
    w1.sub_(lr * w1.grad)
    b1.sub_(lr * b1.grad)
    # w2.sub_(lr * w2.grad)
    # b2.sub_(lr * b2.grad)
    w1.grad=None
    b1.grad=None
    # w2.grad=None
    # b2.grad=None




0 tensor(0.5624, grad_fn=<MeanBackward0>)
1000 tensor(0.0382, grad_fn=<MeanBackward0>)
2000 tensor(0.0242, grad_fn=<MeanBackward0>)
3000 tensor(0.0150, grad_fn=<MeanBackward0>)
4000 tensor(0.0096, grad_fn=<MeanBackward0>)
5000 tensor(0.0064, grad_fn=<MeanBackward0>)
6000 tensor(0.0044, grad_fn=<MeanBackward0>)
7000 tensor(0.0031, grad_fn=<MeanBackward0>)
8000 tensor(0.0023, grad_fn=<MeanBackward0>)
9000 tensor(0.0018, grad_fn=<MeanBackward0>)
10000 tensor(0.0015, grad_fn=<MeanBackward0>)
11000 tensor(0.0012, grad_fn=<MeanBackward0>)
12000 tensor(0.0011, grad_fn=<MeanBackward0>)
13000 tensor(0.0009, grad_fn=<MeanBackward0>)
14000 tensor(0.0008, grad_fn=<MeanBackward0>)
15000 tensor(0.0007, grad_fn=<MeanBackward0>)
16000 tensor(0.0007, grad_fn=<MeanBackward0>)
17000 tensor(0.0006, grad_fn=<MeanBackward0>)
18000 tensor(0.0005, grad_fn=<MeanBackward0>)
19000 tensor(0.0005, grad_fn=<MeanBackward0>)


In [4]:
fig= make_subplots()
fig.add_scatter(x=x.squeeze(),y=y.squeeze(), mode='markers')
fig.add_scatter(x=x.squeeze(), y=predict.detach().numpy(), mode='lines')
fig.update_layout(width=800,height=500)
fig.show()

In [6]:
model = nn.Sequential(nn.Linear(1,20),
                      nn.Tanh(),
                      # nn.Linear(20,20),
                      # nn.Tanh(),
                      nn.Linear(20,1))

loss_fn = nn.MSELoss()
optim = torch.optim.Adam(model.parameters())
for i in range(10000):
  loss = loss_fn(model(x_norm), y)
  if i % 1000 == 0:
    print(i, loss)
  optim.zero_grad()
  loss.backward()
  optim.step()


0 tensor(0.6718, grad_fn=<MseLossBackward>)
1000 tensor(0.1027, grad_fn=<MseLossBackward>)
2000 tensor(0.0237, grad_fn=<MseLossBackward>)
3000 tensor(0.0023, grad_fn=<MseLossBackward>)
4000 tensor(0.0003, grad_fn=<MseLossBackward>)
5000 tensor(3.4513e-05, grad_fn=<MseLossBackward>)
6000 tensor(6.9035e-06, grad_fn=<MseLossBackward>)
7000 tensor(4.2401e-06, grad_fn=<MseLossBackward>)
8000 tensor(3.2492e-06, grad_fn=<MseLossBackward>)
9000 tensor(2.6003e-06, grad_fn=<MseLossBackward>)


In [7]:
fig= make_subplots()
fig.add_scatter(x=x.squeeze(),y=y.squeeze(),mode='markers')
fig.add_scatter(x=x.squeeze(), y=model(x_norm).squeeze().detach().numpy(), mode='lines')
fig.update_layout(width=800,height=500)
fig.show()

In [2]:
transforms=tv.transforms.Compose([tv.transforms.ToTensor()])
mnist_train=tv.datasets.MNIST('./', train=True, download=True, transform=transforms)
mnist_test = tv.datasets.MNIST('./', train=False, download=True, transform=transforms)

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw




Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning:

The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)



In [60]:
sampler = torch.utils.data.DataLoader(mnist_train, batch_size=64, shuffle=True,drop_last=True)
data=iter(sampler)
a=next(data)
pic,label = a

In [61]:

print(pic.shape)
print(label.shape)
img=tv.utils.make_grid(pic, pad_value=255)
print(img.shape)

torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([3, 242, 242])


In [62]:
fig = make_subplots()
fig.add_image(z=img.permute([1,2,0])*255, colormodel='rgb')
fig.update_layout(width=500, height=500, margin=dict(l=2,r=2,b=2,t=2), xaxis_visible=False, yaxis_visible=False)

In [63]:
label.reshape(8,8)

tensor([[6, 5, 9, 8, 9, 4, 7, 4],
        [4, 9, 9, 6, 1, 5, 3, 2],
        [3, 2, 5, 7, 3, 2, 9, 5],
        [8, 0, 3, 3, 0, 4, 7, 7],
        [5, 0, 4, 1, 6, 6, 6, 7],
        [3, 0, 6, 5, 2, 4, 7, 4],
        [6, 6, 4, 2, 5, 2, 4, 1],
        [0, 9, 2, 3, 3, 2, 4, 0]])

In [64]:
model = nn.Sequential(nn.Linear(784,100),
                      nn.Tanh(),
                      nn.Linear(100,100),
                      nn.Tanh(),
                      nn.Linear(100,10),
                      nn.Softmax(dim=0))


In [65]:
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters())
train_sampler = torch.utils.data.DataLoader(mnist_train, batch_size=5000, shuffle=True,drop_last=True)
test_sampler = torch.utils.data.DataLoader(mnist_test, batch_size=5000, shuffle=True,drop_last=True)


for epoch in range(10):
  train_data=iter(train_sampler)
  for i, (train_pic, train_label) in enumerate(train_data):
    optim.zero_grad()
    predict=model(train_pic.flatten(1))
    loss = loss_fn(predict,train_label)
    if i % 5 == 0 :
      print(f'epoch: {epoch} minibatch: {i}  loss: {loss}')
    loss.backward()
    optim.step()
  test_pic, test_label=next(iter(test_sampler))
  predict_label=torch.argmax(model(test_pic.flatten(1)),dim=1)
  print(torch.mean((predict_label==test_label).float()))
  print()



epoch: 0 minibatch: 0  loss: 2.3026669025421143
epoch: 0 minibatch: 5  loss: 2.302598237991333
epoch: 0 minibatch: 10  loss: 2.3024308681488037
tensor(0.7246)

epoch: 1 minibatch: 0  loss: 2.3023438453674316
epoch: 1 minibatch: 5  loss: 2.3021278381347656
epoch: 1 minibatch: 10  loss: 2.3019015789031982
tensor(0.7742)

epoch: 2 minibatch: 0  loss: 2.3018219470977783
epoch: 2 minibatch: 5  loss: 2.3016164302825928
epoch: 2 minibatch: 10  loss: 2.301454782485962
tensor(0.8014)

epoch: 3 minibatch: 0  loss: 2.301391124725342
epoch: 3 minibatch: 5  loss: 2.3012614250183105
epoch: 3 minibatch: 10  loss: 2.3011279106140137
tensor(0.8336)

epoch: 4 minibatch: 0  loss: 2.301081657409668
epoch: 4 minibatch: 5  loss: 2.300999164581299
epoch: 4 minibatch: 10  loss: 2.300941228866577
tensor(0.8438)

epoch: 5 minibatch: 0  loss: 2.3009226322174072
epoch: 5 minibatch: 5  loss: 2.3008971214294434
epoch: 5 minibatch: 10  loss: 2.3008718490600586
tensor(0.8488)

epoch: 6 minibatch: 0  loss: 2.300869464

In [50]:
print(test_label[:100])
print(predict_label[:100])

tensor([9, 2, 3, 9, 8, 2, 7, 9, 7, 7, 5, 4, 4, 7, 2, 0, 7, 7, 7, 0, 3, 9, 5, 8,
        2, 7, 1, 2, 9, 4, 7, 9, 6, 8, 0, 0, 6, 6, 5, 9, 6, 0, 5, 8, 8, 9, 5, 6,
        7, 7, 3, 9, 8, 6, 1, 8, 2, 9, 6, 6, 0, 9, 6, 7, 0, 8, 7, 3, 2, 8, 5, 1,
        3, 9, 9, 1, 5, 9, 8, 7, 8, 4, 4, 0, 8, 6, 9, 7, 0, 2, 5, 5, 0, 2, 8, 2,
        4, 0, 8, 5])
tensor([9, 2, 3, 9, 8, 2, 7, 7, 7, 7, 5, 4, 4, 7, 2, 2, 7, 7, 7, 0, 7, 9, 5, 5,
        2, 7, 1, 2, 9, 4, 7, 9, 6, 8, 0, 0, 6, 6, 5, 1, 6, 0, 5, 5, 8, 9, 5, 6,
        7, 7, 3, 9, 8, 6, 1, 8, 2, 9, 6, 6, 0, 9, 6, 7, 0, 4, 7, 3, 2, 8, 5, 1,
        3, 9, 5, 1, 8, 9, 8, 2, 8, 4, 1, 0, 8, 6, 9, 1, 0, 2, 2, 5, 0, 2, 5, 7,
        4, 0, 8, 8])


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_conv = nn.Sequential(nn.Conv2d(1, 16, 5),
                      nn.MaxPool2d((2,2)),
                      nn.ReLU(),
                      nn.Dropout2d(),
                      nn.Conv2d(16,32,5),
                      nn.MaxPool2d((2,2)),
                      nn.ReLU(),
                      nn.Flatten(1),
                      nn.Linear(512,50),
                      nn.ReLU(),
                      nn.Dropout(),
                      nn.Linear(50,10),
                      nn.Softmax(dim=1)
                      ).to(device)


In [ ]:

loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model_conv.parameters())
train_sampler = torch.utils.data.DataLoader(mnist_train, batch_size=60000, shuffle=True,drop_last=True, pin_memory=True)
test_sampler = torch.utils.data.DataLoader(mnist_test, batch_size=len(mnist_test), shuffle=True,drop_last=True, pin_memory=True)
test_pic, test_label=next(iter(test_sampler))
test_pic = test_pic.to(device)


for epoch in range(1000):
  train_data=iter(train_sampler)
  for i, (train_pic, train_label) in enumerate(train_data):
    train_pic=train_pic.to(device)
    optim.zero_grad()
    predict=model_conv(train_pic).to('cpu')
    loss = loss_fn(predict,train_label)
    # if i % 1 == 0 :
    #   print(f'epoch: {epoch} minibatch: {i}  loss: {loss}')
    loss.backward()
    optim.step()
  predict_label=torch.argmax(model_conv(test_pic).to('cpu'),dim=1)
  print(f'epoch: {epoch} accuracy:{torch.mean((predict_label==test_label).float())}')